In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
import re
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

### Create function to scrape data from domain

In [2]:
class DomainSpider(scrapy.Spider):
    name = 'domain'
    
    def start_requests(self):
        number = self.number
        street = self.street
        suburb = self.suburb
        state = self.state
        postcode = self.postcode
        
        url = f'https://www.domain.com.au/property-profile/{number}-{street}-{suburb}-{state}-{postcode}'
        yield scrapy.Request(url=url, callback=self.parse, meta={
            'number': number,
            'street': street,
            'suburb': suburb,
            'state': state,
            'postcode': postcode
        })
    
    def parse(self, response):
        number = response.meta['number']
        street = response.meta['street']
        suburb = response.meta['suburb']
        state = response.meta['state']
        postcode = response.meta['postcode']
        
        dates = []
        events = []
        prices = []
        
        if response.status != 200:
            dates.append(None)
            events.append(None)
            prices.append(None)
        else:
            property_history_section = response.css('.property-history')
            rows = property_history_section.css('tbody tr')
            
            for row in rows:
                date = row.css('td.date span::text').get()
                event = row.css('td.event::text').get()
                price = row.css('td.price::text').get()
                
                dates.append(date)
                events.append(event)
                prices.append(price)
                
            scraped_data= pd.DataFrame({
                'number': number,
                'street': street,
                'suburb': suburb,
                'state': state,
                'postcode': postcode,
                'dates': dates,
                'events': events,
                'prices': prices
            })
            
        yield scraped_data

### Read in registry data

In [4]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'stra_property_register.csv')

df = pd.read_csv(file_path)
df

,AddressID,TypeOfAccomodation,RegistrantOwnershipStatus,HostingType,FULLADDRESS,PropertyStatus,EndDateCurRenYr,Suburb,PostCode,DeregistrationDate,CouncilName,PropertySubStatus,HostOwnership,DevtConsentNumber,IsDAissued
0,PID-STRA-1,Dual Occupancy,Owner,Non-Hosted,853 CUMBERDOON WAY WALGETT 2832,De-Registered,31/10/2022 23:59,WALGETT,2832.0,4/03/2022 0:00,CUMBERLAND,Sold,Owner,NaN,True
1,PID-STRA-100,Residential Flat Building,Tenant,Hosted,38 45 BONAR STREET ARNCLIFFE 2205,Blocked,31/10/2022 23:59,ARNCLIFFE,2205.0,22/09/2022 0:00,BAYSIDE COUNCIL,Unpaid Renewal Fee,Tenant,NaN,NaN
2,PID-STRA-10000,Secondary Dwelling,Owner,Hosted,63 KITCHENER STREET ST IVES 2075,Registered,31/10/2023 23:59,ST IVES,2075.0,NaN,KU-RING-GAI COUNCIL,Active,Owner,NaN,False
3,PID-STRA-10001,Dwelling House,Owner,Non-Hosted,1 SUNSET STRIP MANYANA 2539,Registered,31/10/2023 23:59,MANYANA,2539.0,NaN,SHOALHAVEN CITY COUNCIL,Active,Owner,NaN,False
4,PID-STRA-1000-1,Dwelling House,Owner,Hosted,117 CUDGERA CREEK ROAD BURRINGBAR 2483,Registered,31/10/2023 23:59,BURRINGBAR,2483.0,NaN,TWEED SHIRE COUNCIL,Active,Owner,DA18/0569.01,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50305,PID-STRA-9995,Semi-Detached Dwelling,Owner,Hosted,23 GOLDING STREET YAMBA 2464,Registered,31/10/2023 23:59,YAMBA,2464.0,NaN,CLARENCE VALLEY COUNCIL,Active,Owner,NaN,False
50306,PID-STRA-9996-2,Secondary Dwelling,Owner,Hosted,3 WASHINGTON STREET TINONEE 2430,Registered,31/10/2023 23:59,TINONEE,2430.0,NaN,MID-COAST COUNCIL,Active,Owner,NaN,False
50307,PID-STRA-9997,Dwelling House,Owner,Hosted,34 ARCHER CRESCENT MARYLAND 2287,Registered,31/10/2023 23:59,MARYLAND,2287.0,31/01/2023 4:38,NEWCASTLE CITY COUNCIL,Active,Owner,NaN,False
50308,PID-STRA-9998,Dwelling House,Owner,Hosted,101 BUNGAN HEAD ROAD NEWPORT 2106,Blocked,31/10/2022 23:59,NEWPORT,2106.0,NaN,NORTHERN BEACHES COUNCIL,Unpaid Renewal Fee,Owner,NaN,False


### Data processing

##### Filter data for non-hosted and registered STRA only

In [6]:
df = df[(df.HostingType == 'Non-Hosted') & (df.PropertyStatus == 'Registered')]
df.reset_index(inplace=True)
addresses = df.FULLADDRESS.str.lower()
addresses.fillna("No Address", inplace=True)

##### Use regex to extract address components

In [7]:
house_number_regex = r'^[a-z]*\s*\#*[0-9]*\,*\s*[lot\s\d+]*\d+(\s|/|\w+)[0-9]*'
street_name_regex = r'\b\w+\s(?:st(?:reet)?|r(?:oa)?d|lane|ln|circuit|dr(?:ive)?|ave(?:nue)?|pl(?:ace)?|boulevard|way|terrace|circle|court|parkway|strip)\b'
# suburb_regex = fr'(?<!{street_name_regex})\s\w+\s\w*\d{4}$'
postcode_regex = r'\d{4}$'

house_numbers = []
street_names = []
postcodes = []

for count, address in enumerate(addresses):
    try:
        house_number = re.search(house_number_regex, address).group().strip()
        house_numbers.append(house_number)
    except:
        house_numbers.append("")
        
for count, address in enumerate(addresses):
    try:
        street_name = re.search(street_name_regex, address).group().strip()
        street_names.append(street_name)
    except:
        street_names.append("")
        continue
        
for address in addresses:
    try:
        postcode = re.search(postcode_regex, address).group().strip()
        postcodes.append(postcode)
    except:
        postcodes.append("")
        continue
    
suburbs = df.Suburb.str.lower()
    
address_processed = pd.DataFrame({
    'house_num': house_numbers,
    'street_name': street_names,
    'suburb': suburbs,
    'postcode': postcodes
})

address_processed

,house_num,street_name,suburb,postcode
0,1,sunset strip,manyana,2539
1,1 230,palmer street,darlinghurst,2010
2,30,cooinda place,kiama,2533
3,305,katoomba street,katoomba,2780
4,96,heath road,pretty beach,2257
...,...,...,...,...
27544,63,sandy place,long beach,2536
27545,72,moncur street,woollahra,2025
27546,1 29,leofrene avenue,marrickville,2204
27547,43,warners avenue,bondi beach,2026


##### Additional processing to ensure data is formatted correctly when passed through the scraping function

In [8]:
address_processed.street_name.replace(['\srd$'], " road", regex=True, inplace=True)
address_processed.street_name.replace(['\sst$'], " street", regex=True, inplace=True)
address_processed.street_name.replace(['\save$'], " avenue", regex=True, inplace=True)
address_processed.street_name.replace(['\sln$'], " lane", regex=True, inplace=True)
address_processed.street_name.replace(['\spl$'], " place", regex=True, inplace=True)

address_processed.street_name.replace(['\s'], "-", regex=True, inplace=True)
address_processed.suburb.replace(['\s'], "-", regex=True, inplace=True)
address_processed.house_num.replace(['\s'], "-", regex=True, inplace=True)

In [9]:
address_processed

,house_num,street_name,suburb,postcode
0,1,sunset-strip,manyana,2539
1,1-230,palmer-street,darlinghurst,2010
2,30,cooinda-place,kiama,2533
3,305,katoomba-street,katoomba,2780
4,96,heath-road,pretty-beach,2257
...,...,...,...,...
27544,63,sandy-place,long-beach,2536
27545,72,moncur-street,woollahra,2025
27546,1-29,leofrene-avenue,marrickville,2204
27547,43,warners-avenue,bondi-beach,2026


### Run the web scraper

In [ ]:
process = CrawlerProcess(get_project_settings())

for i in range(len(address_processed)):
    try:
        process.crawl(DomainSpider, number=address_processed.house_num[i], street=address_processed.street_name[i], suburb=address_processed.suburb[i], state='nsw', postcode=address_processed.postcode[i])
    except Exception as e:
        print(f"Error processing row {i}: {e}")

# start the scrapy engine
process.start()

2023-05-05 19:11:52 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2023-05-05 19:11:52 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.4.0, Python 3.6.8 (default, Apr 16 2020, 01:36:27) - [GCC 8.3.1 20191121 (Red Hat 8.3.1-5)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform Linux-4.18.0-193.14.2.el8_2.x86_64-x86_64-with-centos-8.2.2004-Core
2023-05-05 19:11:52 [scrapy.crawler] INFO: Overridden settings:
{}
2023-05-05 19:11:52 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2023-05-05 19:11:52 [scrapy.extensions.telnet] INFO: Telnet Password: 9df05903dd93fbc4
2023-05-05 19:11:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-05-05 19:11:52 [scrapy.middleware] INFO: En